In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df= pd.read_csv(r"/content/drive/MyDrive/updated_cleaned_data.csv")
df.head()

,document,summary,id,rg_labels,ext_labels
0,hi i I am getting a new laptop i do nt know wh...,of my post is asking what can i do compensate ...,TLDR_RS_2021-04-cm-17760.json,[1.0],[1]
1,it 's possible that many of you already do thi...,if you play hearthstone on mac and it runs poo...,TLDR_RS_2021-02-cm-3714.json,"[0.16403576510432702, 0.35967370810124305, 0.2...","[0, 1, 0, 0]"
2,"i will be the first to admit , i 've got a wei...",i 'm tired of my boyfriend bringing up my weig...,TLDR_RS_2021-04-cm-1732.json,"[0.1865050509103, 0.389592934910708, 0.2526842...","[0, 1, 0, 0, 0]"
3,"hello , my friend 22m broke up a few months ag...",: my friend has suddenly stopped talking to me...,TLDR_RS_2021-03-cm-34926.json,"[0.13981801700034802, 0.054837487497308005, 0....","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
4,background : my 18f ex boyfriend 18m really wa...,my ex is very erratic and depressed and never ...,TLDR_RS_2021-02-cm-18586.json,"[0.18652821177119003, 0.149288690725626, 0.038...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   document    50000 non-null  object
 1   summary     49993 non-null  object
 2   id          50000 non-null  object
 3   rg_labels   50000 non-null  object
 4   ext_labels  50000 non-null  object
dtypes: object(5)
memory usage: 1.9+ MB


In [ ]:
model_df = df.copy()

# Specify the relevant columns in the new dataset
relevant_columns = ['document', 'summary']

# Filter the DataFrame to keep only the relevant columns
model_df = model_df[relevant_columns]

# Display the first few rows of the new DataFrame
model_df.head()

,document,summary
0,hi i I am getting a new laptop i do nt know wh...,of my post is asking what can i do compensate ...
1,it 's possible that many of you already do thi...,if you play hearthstone on mac and it runs poo...
2,"i will be the first to admit , i 've got a wei...",i 'm tired of my boyfriend bringing up my weig...
3,"hello , my friend 22m broke up a few months ag...",: my friend has suddenly stopped talking to me...
4,background : my 18f ex boyfriend 18m really wa...,my ex is very erratic and depressed and never ...


In [ ]:
import torch

# Check if GPU is available and set the device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
import torch
import pandas as pd

# Step 2: Select a smaller subset of the dataset
subset_size = 10000  # Choose the size of your subset
subset_df = df.sample(n=subset_size, random_state=42)  # Randomly select records
#subset_df = df

# Step 3: Split the dataset into training and testing sets
train_df, test_df = train_test_split(subset_df, test_size=0.2, random_state=42)

# Step 4: Load the BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Function to tokenize the text data
def tokenize_function(examples,summary_max_length=150):
    documents = examples['document'].astype(str).tolist()
    summaries = examples['summary'].astype(str).tolist()

    # Tokenize the documents and summaries
    model_inputs = tokenizer(documents, max_length=1024, truncation=True, padding=True)
    labels = tokenizer(summaries, max_length=100, truncation=True, padding=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize the training and test data
train_encodings = tokenize_function(train_df,summary_max_length=150)
test_encodings = tokenize_function(test_df,summary_max_length=150)

In [ ]:
from transformers import AdamW

# Step 4: Prepare the data for the Trainer
class SummaryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = SummaryDataset(train_encodings)
test_dataset = SummaryDataset(test_encodings)

In [ ]:
from transformers import BartForConditionalGeneration

# Load pre-trained BART model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

# Freeze the first 10 encoder layers
for i in range(10):
    for param in model.model.encoder.layers[i].parameters():
        param.requires_grad = False


# Freeze the first 10 decoder layers
for i in range(10):
    for param in model.model.decoder.layers[i].parameters():
        param.requires_grad = False

# Check if the layers are frozen
for i in range(10):
    print(f"Encoder layer {i+1} frozen: {[p.requires_grad for p in model.model.encoder.layers[i].parameters()]}")
    print(f"Decoder layer {i+1} frozen: {[p.requires_grad for p in model.model.decoder.layers[i].parameters()]}")

Encoder layer 1 frozen: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Decoder layer 1 frozen: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Encoder layer 2 frozen: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Decoder layer 2 frozen: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Encoder layer 3 frozen: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
Decoder layer 3 frozen: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Fal

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,                      # Enabled mixed precision
)

# Step 8: Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# Step 9: Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: amanjotkaursohal-1124 (amanjotkaursohal-1124-lambton-college). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.449300,1.377063


Epoch,Training Loss,Validation Loss
1,1.449300,1.377063
2,1.288100,1.351416
3,1.182800,1.359201


TrainOutput(global_step=12000, training_loss=1.3601610565185547, metrics={'train_runtime': 4386.4957, 'train_samples_per_second': 5.471, 'train_steps_per_second': 2.736, 'total_flos': 5.2010510450688e+16, 'train_loss': 1.3601610565185547, 'epoch': 3.0})

In [ ]:
# Save the model
model.save_pretrained('/content/drive/MyDrive/bart_10E10D(10)_model')

# Save the tokenizer
tokenizer.save_pretrained('/content/drive/MyDrive/bart_10E10D(10)_tokenizer')

('/content/drive/MyDrive/bart_10E10D(10)_tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/bart_10E10D(10)_tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/bart_10E10D(10)_tokenizer/vocab.json',
 '/content/drive/MyDrive/bart_10E10D(10)_tokenizer/merges.txt',
 '/content/drive/MyDrive/bart_10E10D(10)_tokenizer/added_tokens.json')

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # Add this if it prompts for 'punkt_tab'


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
import nltk
from sklearn.metrics import f1_score
from transformers import BertTokenizer

# Download necessary NLTK data files
nltk.download('punkt')

# Initialize ROUGE scorer
rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Define device and move model to that device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Initialize BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Function to generate summaries
def generate_summary(text, max_length=150):
    model.eval()  # Set model to evaluation mode

    # Tokenize the input and move to the correct device
    inputs = tokenizer(text, max_length=1024, truncation=True, return_tensors="pt").to(device)

    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=max_length, num_beams=4, length_penalty=2.0)

    # Decode the generated summary and return
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Calculate ROUGE, BLEU, and BERT F1 scores
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
bleu_scores = []
bert_f1_scores = []

for idx, row in test_df.iterrows():
    # Generate model summary
    generated_summary = generate_summary(row['document'])

    # Reference summary
    reference_summary = row['summary']

    # Calculate ROUGE scores
    scores = rouge_scorer.score(reference_summary, generated_summary)
    rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
    rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
    rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

    # Calculate BLEU score
    reference_tokens = nltk.word_tokenize(reference_summary)
    generated_tokens = nltk.word_tokenize(generated_summary)
    bleu_score = sentence_bleu([reference_tokens], generated_tokens)
    bleu_scores.append(bleu_score)

    # Tokenize using BERT tokenizer
    reference_tokens_bert = bert_tokenizer.tokenize(reference_summary)
    generated_tokens_bert = bert_tokenizer.tokenize(generated_summary)

    # Calculate BERT F1 score
    # First calculate precision and recall based on token matches
    common_tokens = set(reference_tokens_bert) & set(generated_tokens_bert)
    precision = len(common_tokens) / len(generated_tokens_bert) if len(generated_tokens_bert) > 0 else 0
    recall = len(common_tokens) / len(reference_tokens_bert) if len(reference_tokens_bert) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    bert_f1_scores.append(f1)

# Calculate average scores
avg_rouge1 = sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1'])
avg_rouge2 = sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2'])
avg_rougeL = sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL'])
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_bert_f1 = sum(bert_f1_scores) / len(bert_f1_scores)

# Print evaluation results
print(f"Average ROUGE-1 Score: {avg_rouge1:.4f}")
print(f"Average ROUGE-2 Score: {avg_rouge2:.4f}")
print(f"Average ROUGE-L Score: {avg_rougeL:.4f}")
print(f"Average BLEU Score: {avg_bleu:.4f}")
print(f"Average BERT F1 Score: {avg_bert_f1:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Average ROUGE-1 Score: 0.2679
Average ROUGE-2 Score: 0.0825
Average ROUGE-L Score: 0.1972
Average BLEU Score: 0.0235
Average BERT F1 Score: 0.2425
